# Hobohm 1

### Fill in the blanked out part of the code (XX)

## Python Imports

In [12]:
import numpy as np
import math 
from time import time

## Data Imports

## DEFINE THE PATH TO YOUR COURSE DIRECTORY

In [13]:
data_dir = "/home/mathias/bioinfo_algos/data/"

In [14]:
alphabet_file = data_dir + "Matrices/alphabet"
alphabet = np.loadtxt(alphabet_file, dtype=str)

alphabet

blosum_file = data_dir + "Matrices/BLOSUM50"
_blosum50 = np.loadtxt(blosum_file, dtype=int).T

blosum50 = {}

for i, letter_1 in enumerate(alphabet):
    
    blosum50[letter_1] = {}

    for j, letter_2 in enumerate(alphabet):
        
        blosum50[letter_1][letter_2] = _blosum50[i, j]
        
blosum50      

{'A': {'A': 5,
  'R': -2,
  'N': -1,
  'D': -2,
  'C': -1,
  'Q': -1,
  'E': -1,
  'G': 0,
  'H': -2,
  'I': -1,
  'L': -2,
  'K': -1,
  'M': -1,
  'F': -3,
  'P': -1,
  'S': 1,
  'T': 0,
  'W': -3,
  'Y': -2,
  'V': 0},
 'R': {'A': -2,
  'R': 7,
  'N': -1,
  'D': -2,
  'C': -4,
  'Q': 1,
  'E': 0,
  'G': -3,
  'H': 0,
  'I': -4,
  'L': -3,
  'K': 3,
  'M': -2,
  'F': -3,
  'P': -3,
  'S': -1,
  'T': -1,
  'W': -3,
  'Y': -1,
  'V': -3},
 'N': {'A': -1,
  'R': -1,
  'N': 7,
  'D': 2,
  'C': -2,
  'Q': 0,
  'E': 0,
  'G': 0,
  'H': 1,
  'I': -3,
  'L': -4,
  'K': 0,
  'M': -2,
  'F': -4,
  'P': -2,
  'S': 1,
  'T': 0,
  'W': -4,
  'Y': -2,
  'V': -3},
 'D': {'A': -2,
  'R': -2,
  'N': 2,
  'D': 8,
  'C': -4,
  'Q': 0,
  'E': 2,
  'G': -1,
  'H': -1,
  'I': -4,
  'L': -4,
  'K': -1,
  'M': -4,
  'F': -5,
  'P': -1,
  'S': 0,
  'T': -1,
  'W': -5,
  'Y': -3,
  'V': -4},
 'C': {'A': -1,
  'R': -4,
  'N': -2,
  'D': -4,
  'C': 13,
  'Q': -3,
  'E': -3,
  'G': -3,
  'H': -3,
  'I': -2,
  'L'

### Sequences

In [15]:
def load_sequences():
    
    database_file = data_dir + "Hobohm/database_list.tab"
    database_list = np.loadtxt(database_file, dtype=str).reshape(-1,2)

    ids = database_list[:, 0]
    sequences = database_list[:, 1]

    return sequences, ids

## Smith-Waterman O2

### This code is identical to the code you wrote the other day

In [16]:
def smith_waterman(query, database, scoring_scheme, gap_open, gap_extension):
    
    P_matrix, Q_matrix, D_matrix, E_matrix, i_max, j_max, max_score = smith_waterman_alignment(query, database, scoring_scheme, gap_open, gap_extension)
    
    aligned_query, aligned_database, matches = smith_waterman_traceback(E_matrix, D_matrix, i_max, j_max, query, database, gap_open, gap_extension)
    
    return aligned_query, aligned_database, matches


def smith_waterman_alignment(query, database, scoring_scheme, gap_open, gap_extension):

    # Matrix imensions
    M = len(query)
    N = len(database)
    
    # D matrix change to float
    D_matrix = np.zeros((M+1, N+1), int)

    # P matrix
    P_matrix = np.zeros((M+1, N+1), int)
    
    # Q matrix
    Q_matrix = np.zeros((M+1, N+1), int)

    # E matrix
    E_matrix = np.zeros((M+1, N+1), dtype=object)

    # Main loop
    D_matrix_max_score, D_matrix_i_max, D_matrix_i_max = -9, -9, -9
    for i in range(M-1, -1, -1):
        for j in range(N-1, -1, -1):
            
            # Q_matrix[i,j] entry
            gap_open_database = D_matrix[i+1,j] + gap_open
            gap_extension_database = Q_matrix[i+1,j] + gap_extension
            max_gap_database = max(gap_open_database, gap_extension_database)
            
            Q_matrix[i,j] = max_gap_database
                
            # P_matrix[i,j] entry
            gap_open_query = D_matrix[i,j+1] + gap_open
            gap_extension_query = P_matrix[i,j+1] + gap_extension
            max_gap_query = max(gap_open_query, gap_extension_query)
            
            P_matrix[i,j] = max_gap_query
            
            # D_matrix[i,j] entry
            diagonal_score = D_matrix[i+1,j+1] + scoring_scheme[query[i]][database[j]]    
            
            # E_matrix[i,j] entry
            candidates = [(1, diagonal_score),
                          (2, gap_open_database),
                          (4, gap_open_query),
                          (3, gap_extension_database),
                          (5, gap_extension_query)]
            
            direction, max_score = max(candidates, key=lambda x: x[1])
            
            
            # check entry sign
            if max_score > 0:
                E_matrix[i,j] = direction
            else:
                E_matrix[i,j] = 0
            
            # check max score sign
            if max_score > 0:
                D_matrix[i, j] = max_score
            else:
                D_matrix[i, j] = 0

            # fetch global max score
            if max_score > D_matrix_max_score:
                D_matrix_max_score = max_score
                D_matrix_i_max = i
                D_matrix_j_max = j
            
    return P_matrix, Q_matrix, D_matrix, E_matrix, D_matrix_i_max, D_matrix_j_max, D_matrix_max_score


def smith_waterman_traceback(E_matrix, D_matrix, i_max, j_max, query, database, gap_open, gap_extension):
    
    # Matrix imensions
    M = len(query)
    N = len(database)
    
    # aligned query string
    aligned_query = []
    
    # aligned database string
    aligned_database = []
    
    # total identical matches
    matches = 0

        
    # start from max_i, max_j
    i, j = i_max, j_max
    while i < M and j < N:

        # E[i,j] = 0, stop back tracking
        if E_matrix[i, j] == 0:
            break
        
        # E[i,j] = 1, match
        if E_matrix[i, j] == 1:
            aligned_query.append(query[i])
            aligned_database.append(database[j])
            if ( query[i] == database[j]):
                matches += 1
            i += 1
            j += 1
        
        
        # E[i,j] = 2, gap opening in database
        if E_matrix[i, j] == 2:
            aligned_database.append("-")
            aligned_query.append(query[i])
            i += 1

            
        # E[i,j] = 3, gap extension in database
        if E_matrix[i, j] == 3:
                   
            count = i + 2
            score = D_matrix[count, j] + gap_open + gap_extension
            
            # Find length of gap
            while((score - D_matrix[i, j])*(score - D_matrix[i, j]) >= 0.00001):   
                count += 1
                score = D_matrix[count, j] + gap_open + (count-i-1)*gap_extension

            for k in range(i, count):
                aligned_database.append("-")
                aligned_query.append(query[i])
                i += 1
            
            
        # E[i,j] = 4, gap opening in query
        if E_matrix[i, j] == 4:
            aligned_query.append("-")
            aligned_database.append(database[j])
            j += 1
        
        
        # E[i,j] = 5, gap extension in query
        if E_matrix[i, j] == 5:
             
            count = j + 2
            score = D_matrix[i, count] + gap_open + gap_extension
            
            # Find length of gap
            while((score - D_matrix[i, j])*(score - D_matrix[i, j]) >= 0.0001): 
                count += 1
                score = D_matrix[i, count] + gap_open + (count-j-1)*gap_extension

            for k in range(j, count):
                aligned_query.append("-")
                aligned_database.append(database[j])
                j += 1

                
    return aligned_query, aligned_database, matches

## Hobohm 1

### Similarity Function

### This code defines the threshold for similarity

In [17]:
def homology_function(alignment_length, matches):

    homology_score = 2.9*math.sqrt(alignment_length)
    
    if matches > homology_score: ## Add the inequally sign
        return "discard", homology_score
    else:
        return "keep", homology_score

### Main Loop

In [18]:
# load list
candidate_sequences, candidate_ids = load_sequences()
print ("# Number of elements:", len(candidate_sequences))

accepted_sequences, accepted_ids = [], []

accepted_sequences.append(candidate_sequences[0])
accepted_ids.append(candidate_ids[0])

print ("# Unique.", 0, len(accepted_sequences)-1, accepted_ids[0])

# parameters
scoring_scheme = blosum50
gap_open = -11
gap_extension = -1

t0 = time()

for i in range(1, len(candidate_sequences)):
             
    for j in range(0, len(accepted_sequences)):
        
        query = candidate_sequences[i]
        database = accepted_sequences[j]
        
        aligned_query, aligned_database, matches = smith_waterman(query, database, scoring_scheme, gap_open, gap_extension)
        
        alignment_length = len(aligned_query)
        
        homology_outcome, homology_score = homology_function(alignment_length, matches)
     
        # query is not unique
        if homology_outcome == "discard":

            print ("# Not unique.", i, candidate_ids[i], "is homolog to", accepted_ids[j], homology_score)
            
            break
            
            
    # query is unique
    if homology_outcome == "keep":
        accepted_sequences.append(candidate_sequences[i])
        accepted_ids.append(candidate_ids[i])

        print ("# Unique.", i, len(accepted_sequences)-1, candidate_ids[i], homology_score)

t1 = time()

print ("Elapsed time (m):", (t1-t0)/60)

print ("Accepted sequences:", len(accepted_ids))
for i in range(len(accepted_ids)):
    print (accepted_ids[i])


# Number of elements: 1052
# Unique. 0 0 1US0.A
# Not unique. 1 2EAB.A is homolog to 1US0.A 50.89479344687431
# Not unique. 2 1QWN.A is homolog to 1US0.A 52.199999999999996
# Not unique. 3 1HLR.A is homolog to 1US0.A 41.72373425282066
# Not unique. 4 1MXT.A is homolog to 1US0.A 36.682420857953204
# Unique. 5 1 1MUW.A 39.97374138106164
# Not unique. 6 1SU8.A is homolog to 1US0.A 44.926606816006036
# Not unique. 7 1WUI.L is homolog to 1US0.A 44.171484014010666
# Not unique. 8 2BHU.A is homolog to 1MUW.A 49.724641778498516
# Not unique. 9 2E7Z.A is homolog to 1US0.A 43.20902683467888
# Not unique. 10 1JZ7.A is homolog to 1US0.A 60.27536810339693
# Not unique. 11 1GCI._ is homolog to 1US0.A 36.682420857953204
# Not unique. 12 1PJX.A is homolog to 1US0.A 42.81798687467686
# Not unique. 13 1LQT.A is homolog to 1MUW.A 52.60123572692945
# Not unique. 14 1KWF.A is homolog to 1US0.A 42.81798687467686
# Not unique. 15 1MRO.A is homolog to 1US0.A 42.81798687467686
# Not unique. 16 1D5T.A is homolo

# Not unique. 132 1QOP.B is homolog to 1US0.A 52.44110982807286
# Not unique. 133 2DCF.A is homolog to 1US0.A 56.15825852000754
# Not unique. 134 2UYT.A is homolog to 1US0.A 32.42298567374695
# Not unique. 135 1F5N.A is homolog to 1US0.A 41.216744170300494
# Not unique. 136 1ITW.A is homolog to 1MUW.A 49.724641778498516
# Not unique. 137 1KXQ.A is homolog to 1US0.A 52.68111616129635
# Not unique. 138 1ZHX.A is homolog to 1US0.A 49.55522172284167
# Not unique. 139 1X6Z.A is homolog to 1MUW.A 35.988053573373485
# Not unique. 140 1ODZ.A is homolog to 1MUW.A 53.708751614611195
# Not unique. 141 1RLL.A is homolog to 1US0.A 52.99943395924149
# Not unique. 142 1I19.A is homolog to 1MUW.A 60.83091319386879
# Not unique. 143 1Q7Z.A is homolog to 1US0.A 50.646322670061636
# Not unique. 144 1QWO.A is homolog to 1US0.A 51.469894112966664
# Not unique. 145 1NOF.A is homolog to 1MUW.A 45.02010661915407
# Not unique. 146 1EEX.A is homolog to 1US0.A 52.840514759036935
# Not unique. 147 1AOP._ is homol

# Not unique. 266 1PEV.A is homolog to 1US0.A 43.01395122515484
# Not unique. 267 1PMI._ is homolog to 1US0.A 50.061761854733
# Not unique. 268 1SG4.B is homolog to 1US0.A 50.56322774507181
# Not unique. 269 2DPL.A is homolog to 1US0.A 47.74002513614755
# Not unique. 270 1O9I.A is homolog to 1US0.A 46.761094940131585
# Not unique. 271 1RRM.A is homolog to 1US0.A 46.490536671456056
# Not unique. 272 1VCL.A is homolog to 1MUW.A 57.1233752504174
# Not unique. 273 1KRH.A is homolog to 1US0.A 34.920624278497655
# Not unique. 274 1ME3.A is homolog to 2F01.A 34.190495755399624
# Not unique. 275 1P30.A is homolog to 1US0.A 53.63040182583009
# Not unique. 276 1M0W.B is homolog to 1MUW.A 43.01395122515484
# Not unique. 277 2DDR.A is homolog to 1US0.A 50.313119561402665
# Not unique. 278 1Q6O.B is homolog to 1UWC.A 38.14354467010112
# Not unique. 279 1GTT.A is homolog to 1US0.A 53.943118189440995
# Unique. 280 19 1Y93.A 15.61697794069006
# Not unique. 281 1GES.B is homolog to 1MUW.A 52.8405147590

# Not unique. 400 2FW0.A is homolog to 1MUW.A 42.124933234368456
# Not unique. 401 2FAO.A is homolog to 1MUW.A 51.05976889881113
# Not unique. 402 2PXX.A is homolog to 1X8P.A 40.90953434103106
# Not unique. 403 1YC5.A is homolog to 1US0.A 43.306235116897426
# Unique. 404 35 1WC2.A 15.068842025849232
# Not unique. 405 1RT8.A is homolog to 1US0.A 53.47335785229874
# Not unique. 406 1QCX.A is homolog to 1US0.A 45.94464060148909
# Not unique. 407 1MF7.A is homolog to 1MUW.A 39.65690356041429
# Not unique. 408 1ECF.B is homolog to 1US0.A 51.55152762043041
# Not unique. 409 1Z2U.A is homolog to 1US0.A 38.14354467010112
# Unique. 410 36 1P6O.B 30.137684051698464
# Not unique. 411 2GGO.A is homolog to 1MUW.A 57.1233752504174
# Not unique. 412 2BKX.A is homolog to 1US0.A 50.229473419497445
# Not unique. 413 1O97.D is homolog to 1MUW.A 48.00354153601586
# Not unique. 414 2F48.B is homolog to 1US0.A 39.23047284955918
# Not unique. 415 1U6G.C is homolog to 1US0.A 51.38813092534111
# Not unique. 41

# Not unique. 534 1GUP.A is homolog to 1US0.A 44.266578815173865
# Not unique. 535 2AHX.A is homolog to 1US0.A 45.669245669268506
# Not unique. 536 1G4M.A is homolog to 1US0.A 51.14205705679035
# Not unique. 537 2Z1E.A is homolog to 1MUW.A 55.176263012277296
# Not unique. 538 1RP0.A is homolog to 1US0.A 37.36388630750287
# Not unique. 539 2BK9.A is homolog to 1US0.A 37.811373950175366
# Not unique. 540 1Z3X.A is homolog to 1MUW.A 42.22463735782701
# Not unique. 541 2BVC.A is homolog to 1US0.A 50.39662687125002
# Not unique. 542 1LU4.A is homolog to 1UWC.A 38.5819906173852
# Not unique. 543 2I0O.A is homolog to 1MUW.A 52.52123380119702
# Not unique. 544 1TKE.A is homolog to 1MUW.A 36.104708834167326
# Unique. 545 48 1LKK.A 24.263140769488192
# Not unique. 546 1AX4.A is homolog to 1MUW.A 50.14568775079269
# Not unique. 547 1ZV9.A is homolog to 1US0.A 37.47625915162825
# Not unique. 548 1J98.A is homolog to 1MUW.A 36.682420857953204
# Not unique. 549 2GQP.A is homolog to 1US0.A 39.3375139

# Not unique. 667 1TPZ.A is homolog to 1US0.A 48.0910594601533
# Unique. 668 60 1IQZ.A 11.231651704001509
# Not unique. 669 1RKU.A is homolog to 1US0.A 43.88496325622251
# Not unique. 670 2I9U.A is homolog to 1US0.A 43.01395122515484
# Not unique. 671 2E4U.B is homolog to 1US0.A 41.92481365492279
# Not unique. 672 2HSJ.D is homolog to 1US0.A 38.5819906173852
# Not unique. 673 1PGS._ is homolog to 1MUW.A 54.40882281395179
# Unique. 674 61 1F86.A 16.65923167496028
# Not unique. 675 2OZG.A is homolog to 1US0.A 51.63303206281808
# Not unique. 676 1Q7L.A is homolog to 1MUW.A 41.42028488554853
# Not unique. 677 1HW1.A is homolog to 1US0.A 44.266578815173865
# Not unique. 678 1EON.A is homolog to 1MUW.A 43.5
# Not unique. 679 2F6U.A is homolog to 1EB6.A 35.16063139364821
# Not unique. 680 1V7R.A is homolog to 1US0.A 28.70853531617383
# Not unique. 681 2BW8.A is homolog to 1MUW.A 45.2065260775477
# Not unique. 682 2DH2.A is homolog to 1US0.A 50.47999603803471
# Not unique. 683 1I12.D is homolo

# Not unique. 801 1X31.B is homolog to 1MUW.A 44.266578815173865
# Unique. 802 76 1ZZK.A 21.70161284328886
# Not unique. 803 2H8F.B is homolog to 1XJU.A 28.41408101628486
# Not unique. 804 1Y43.B is homolog to 1QTW.A 36.79687486730361
# Unique. 805 77 1I8O.A 14.787156589419075
# Unique. 806 78 1D4T.A 16.65923167496028
# Not unique. 807 1HL9.A is homolog to 1US0.A 51.30623743756698
# Not unique. 808 2GZ4.A is homolog to 1US0.A 43.980677575498994
# Not unique. 809 2O3T.A is homolog to 1US0.A 47.82802525716486
# Not unique. 810 2H7G.X is homolog to 1US0.A 51.795656188526074
# Not unique. 811 1T6K.A is homolog to 1MUW.A 47.208579728689145
# Not unique. 812 2GRR.B is homolog to 1X8P.A 35.75360121722006
# Not unique. 813 2PW4.A is homolog to 1US0.A 34.920624278497655
# Not unique. 814 1II5.A is homolog to 1US0.A 44.456158178592084
# Unique. 815 79 1OK0.A 17.4
# Not unique. 816 1H30.A is homolog to 1US0.A 45.02010661915407
# Not unique. 817 1YNS.A is homolog to 1MUW.A 48.87166868442288
# Not 

# Not unique. 937 1QLM.A is homolog to 1US0.A 45.11341263970173
# Not unique. 938 2DU3.A is homolog to 1US0.A 58.79209130486855
# Not unique. 939 1YPY.A is homolog to 1UWC.A 40.07879738714723
# Not unique. 940 1C8U.A is homolog to 1MUW.A 52.199999999999996
# Not unique. 941 2C5Q.E is homolog to 1US0.A 43.59655949728143
# Not unique. 942 1NLN.A is homolog to 2HC1.A 40.90953434103106
# Not unique. 943 1Z2L.A is homolog to 1MUW.A 56.083330856859774
# Unique. 944 95 1VBW.A 21.50697561257742
# Not unique. 945 1FI2.A is homolog to 1MUW.A 42.81798687467686
# Unique. 946 96 1C5E.A 19.881398341163028
# Not unique. 947 1VHT.A is homolog to 1US0.A 40.392326003833944
# Not unique. 948 1FW8.A is homolog to 1US0.A 49.55522172284167
# Unique. 949 97 1G6X.A 7.103520254071215
# Not unique. 950 1MRZ.B is homolog to 1MUW.A 47.91586376138908
# Not unique. 951 1TXK.A is homolog to 1US0.A 53.63040182583009
# Not unique. 952 1H4X.A is homolog to 1XOD.B 29.8573274088623
# Not unique. 953 2FEF.A is homolog to 